# Tutorial 3 - A Feedforward Spiking Neural Network

## Learn hot to
- simplify the LIF neuron to make it deep learning-friendly
- implement a feedforward SNN

> 실제 딥러닝에 적용하기 위해 LIF 뉴런 모델을 간소화하는 것과, 순전파 SNN을 구현해본다.

---

In [2]:
import snntorch as snn
import torch
import matplotlib.pyplot as plt

from snntorch import spikeplot as splt
from snntorch import spikegen

In [3]:
def LeakyIntegrate_and_Fire(prev_mem, x, w, beta, threshold=1):
    # prev_mem : previous membrane output
    # x        : current input signal
    # w        : weight for current input
    # beta     : decay rate
    # threshold: neuron threshold for spike output
    
    spk = (prev_mem > threshold)     # If membrane exceeds threshold, spk = 1, else = 0
    mem = beta * prev_mem + w*x - spk*threshold
    return spk, mem

In [4]:
# Set neuronal params
delta_t = torch.tensor(1e-3)
tau     = torch.tensor(5e-3)
beta    = torch.exp(-delta_t/tau)

print(f'The decay rate is: {beta:.3f}')

The decay rate is: 0.819
